# 표 형식 특성 벡터화하기

이 노트북은 다른 예제와 동일한 테이터셋을 사용해 표 형태 데이터를 벡터화하는 간단한 방법을 다룹니다. 하지만 이번에는 질문 텍스트를 무시합니다.

In [5]:
def get_normalized_series(df, col):
    """
    DataFrame 열을 정규화합니다.
    :param df: DataFrame
    :param col: 열 이름
    :return: Z-점수를 사용해 정규화된 시리즈 객체
    """
    return (df[col] - df[col].mean()) / df[col].std()

In [2]:
import pandas as pd
from pathlib import Path
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')

data_path = Path("data/writers.csv")
df = pd.read_csv(data_path)

태그, 코멘트 개수, 질문 날짜로부터 **점수**를 예측한다고 가정해 보죠. 이 데이터는 다음과 같습니다.

In [3]:
df["is_question"] = df["PostTypeId"] == 1

tabular_df = df[df["is_question"]][["Tags", "CommentCount", "CreationDate", "Score"]]
tabular_df.head()

,Tags,CommentCount,CreationDate,Score
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,20
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,34
3,<plot><short-story><planning><brainstorming>,0,2010-11-18T20:43:59.693,28
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,21


이 데이터를 모델의 입력으로 사용하기 위해 적절한 수치 표현으로 바꾸어야 합니다. 이렇게 하기 위해 다음 세 가지 작업을 합니다:

1. 수치 입력 특성을 정규화하여 이상치로 인한 영향을 줄입니다.
2. 날짜 특성을 모델이 이해하기 쉬운 형태로 변환합니다.
3. 모델이 범주형 특성을 이해할 수 있도록 더미(dummy) 변수로 바꿉니다.

먼저 데이터를 정규화하여 이상치가 모델 성능에 미치는 영향을 줄입니다.

In [6]:
tabular_df["NormComment"]= get_normalized_series(tabular_df, "CommentCount")
tabular_df["NormScore"]= get_normalized_series(tabular_df, "Score")

In [7]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,1.405553,3.66092
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,20,-0.878113,2.02388
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,34,-0.551875,3.93376
3,<plot><short-story><planning><brainstorming>,0,2010-11-18T20:43:59.693,28,-0.878113,3.11524
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,21,-0.551875,2.16030


이제 날짜를 모델이 패턴을 추출하기 쉬운 형태로 표현해 보죠(특정 특성을 선택한 이유에 대한 자세한 내용은 책의 4장을 참고하세요).

In [8]:
# 날짜를 판다스 datetime으로 변환합니다.
tabular_df["date"] = pd.to_datetime(tabular_df["CreationDate"])

# datetime 객체에서 의미있는 특성을 추출합니다.
tabular_df["year"] = tabular_df["date"].dt.year
tabular_df["month"] = tabular_df["date"].dt.month
tabular_df["day"] = tabular_df["date"].dt.day
tabular_df["hour"] = tabular_df["date"].dt.hour

In [9]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore,date,year,month,day,hour
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,1.405553,3.66092,2010-11-18 20:40:32.857,2010,11,18,20
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,20,-0.878113,2.02388,2010-11-18 20:42:31.513,2010,11,18,20
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,34,-0.551875,3.93376,2010-11-18 20:43:28.903,2010,11,18,20
3,<plot><short-story><planning><brainstorming>,0,2010-11-18T20:43:59.693,28,-0.878113,3.11524,2010-11-18 20:43:59.693,2010,11,18,20
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,21,-0.551875,2.16030,2010-11-18 20:45:44.067,2010,11,18,20


마지막으로 태그를 판다스의 [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) 함수를 사용해 더비 변수로 변환합니다. 각 태그는 하나의 더비 변수에 할당됩니다.

> Tags에 데해서 one-hot 인코딩을 하는 과정

In [14]:
# 문자열로 표현된 태그를 태그 배열로 변환합니다.
tags = tabular_df["Tags"]
clean_tags = tags.str.split("><").apply(
    lambda x: [a.strip("<").strip(">") for a in x])

# 판다스의 get_dummies 함수를 사용해 더미 변수를 만듭니다.
# 500번 이상 나타난 태그만 선택합니다.
tag_columns = pd.get_dummies(clean_tags.apply(pd.Series).stack()).sum(level=0)
all_tags = tag_columns.astype(bool).sum(axis=0).sort_values(ascending=False)
top_tags = all_tags[all_tags > 500]
top_tag_columns = tag_columns[top_tags.index]

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [11]:
top_tag_columns.head()

NameError: name 'top_tag_columns' is not defined

In [ ]:
# 태그를 원래 DateFrame에 추가합니다.
final = pd.concat([tabular_df, top_tag_columns], axis=1)

# 벡터화된 특성만 남깁니다.
col_to_keep = ["year", "month", "day", "hour", "NormComment",
               "NormScore"] + list(top_tags.index)
final_features = final[col_to_keep]

In [ ]:
final_features.head()

좋습니다. 이제 모델에 사용할 데이터가 준비되었습니다.